In [20]:
import os
os.environ["LANGCHAIN_TRACING_V2"] ="true"


In [21]:
import getpass
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()


In [22]:
from langchain_community.llms import Ollama

llm = Ollama(model="mistral", temperature=0)

In [ ]:
llm.invoke("what are the three types of efficient markets hypothesis")

" The Efficient Markets Hypothesis (EMH) suggests that financial markets are informationally efficient, meaning that asset prices reflect all available information. There are three main forms of the EMH:\n\n1. Weak Form Efficiency: This form states that current asset prices already reflect all past price and volume data, i.e., it is impossible to consistently earn abnormal returns by using historical market data for stock selection or market timing. In other words, technical analysis based on past trends and patterns has no predictive power.\n\n2. Semi-Strong Form Efficiency: This form extends the weak form efficiency to include all publicly available information, such as financial statements, analyst reports, and company news. If a security's price reflects all public information, it is impossible to earn abnormal returns by using this information for stock selection or market timing.\n\n3. Strong Form Efficiency: This form suggests that asset prices reflect all publicly available inf

In [24]:
"""
from langchain_community.document_loaders import TextLoader

loader = TextLoader(file_path="data/alice_in_wonderland.md")

docs = loader.load()
"""

In [55]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/Users/osbert/Desktop/inv.pdf")
docs = loader.load()

Ignoring wrong pointing object 289 0 (offset 0)
Ignoring wrong pointing object 736 0 (offset 0)
Ignoring wrong pointing object 1797 0 (offset 0)
Ignoring wrong pointing object 1930 0 (offset 0)
Ignoring wrong pointing object 2204 0 (offset 0)
Ignoring wrong pointing object 2665 0 (offset 0)
Ignoring wrong pointing object 3132 0 (offset 0)
Ignoring wrong pointing object 3241 0 (offset 0)
Ignoring wrong pointing object 3526 0 (offset 0)
Ignoring wrong pointing object 3542 0 (offset 0)
Ignoring wrong pointing object 3608 0 (offset 0)
Ignoring wrong pointing object 4034 0 (offset 0)
Ignoring wrong pointing object 4076 0 (offset 0)
Ignoring wrong pointing object 4555 0 (offset 0)
Ignoring wrong pointing object 5126 0 (offset 0)
Ignoring wrong pointing object 5363 0 (offset 0)
Ignoring wrong pointing object 6277 0 (offset 0)
Ignoring wrong pointing object 6291 0 (offset 0)
Ignoring wrong pointing object 7018 0 (offset 0)
Ignoring wrong pointing object 7602 0 (offset 0)
Ignoring wrong pointin

In [63]:
len(docs[0].page_content)

0

In [109]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 5000,
    chunk_overlap = 500,
    add_start_index = True, # add meta data attribute
)

In [110]:
# apply class to text

all_splits = text_splitter.split_documents(docs)

In [111]:
len(all_splits)

1070

In [112]:
len(all_splits[1].page_content)

3609

In [113]:
#embed each content

from langchain_ollama.embeddings import OllamaEmbeddings

embedding = OllamaEmbeddings(
    model="nomic-embed-text"
    )

In [115]:
from langchain_community.vectorstores import Chroma

# to index
vectore_store = Chroma.from_documents(
    documents = all_splits,
    embedding = embedding,
)

In [116]:
retriever = vectore_store.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":6}
)

In [117]:
retriever.get_relevant_documents(
    "what is efficient markets hypothesis"
)

[Document(metadata={'page': 12, 'source': '/Users/osbert/Desktop/inv.pdf', 'start_index': 0}, page_content='/one.tnum/two.tnum PART I  Introduction\nbod12665_ch01_001-030.indd 12 06/06/22  07:58 AM\nif returns were independent of risk, there would be a rush to sell high-risk assets and \ntheir prices would fall. The assets would get cheaper (improving their expected future \nrates of return) until they eventually were attractive enough to be included again in \ninvestor portfolios. We conclude that there should be a risk–return trade-off  in the \nsecurities markets, with higher-risk assets priced to offer higher expected returns than \nlower-risk assets.\nOf course, this discussion leaves several important questions unanswered. How should \none measure the risk of an asset? What should be the quantitative trade-off between risk \n(properly measured) and expected return? One would think that risk would have some -\nthing to do with the volatility of an asset’s returns, but this guess t

In [118]:
# Generate Output into a chain
# retrieve document
# pass through prompt
# parse the output

from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [119]:
def format_docs(docs):
    return"\n\n". join(doc.page_content for doc in docs)

In [120]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# To generate an output based on our input we created a chain
# paste chain to input to local model to Stroutput parser
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough() }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke ("How might the incentive fee of a hedge fund affect the manager proclivity to take on high-risk assets in the portfolio")

' The text provides an overview of various investment vehicles, including mutual funds, commingled funds, real estate investment trusts (REITs), hedge funds, and money market funds.\n\n1. Mutual Funds: These are investment companies that pool money from many investors to purchase a diversified portfolio of stocks, bonds, or other securities. They offer shares to the public and manage these shares according to a specific investment strategy.\n\n2. Commingled Funds: Similar to open-end mutual funds, commingled funds are partnerships of investors that pool funds. The management firm manages the funds for a fee. Typical partners in a commingled fund might be trust or retirement accounts with portfolios much larger than those of most individual investors, but still too small to warrant managing on a separate basis.\n\n3. Real Estate Investment Trusts (REITs): REITs invest in real estate or loans secured by real estate. They raise capital by borrowing from banks and issuing bonds or mortgage